# Chatbot RAG para Guaraní - Idioma de Bajo Recursos

Este notebook implementa un sistema de chatbot con RAG (Retrieval-Augmented Generation) para el idioma guaraní.

## Objetivos:
1. Comparar 2 modelos de LLM (GPT-3.5 Turbo vs Claude 3.5 Sonnet)
2. Evaluar rendimiento con y sin RAG
3. Comparar few-shot, zero-shot y RAG
4. Evaluar si el idioma de bajo recursos se beneficia del RAG

## Estructura:
- Instalación de dependencias
- Carga de datos desde PDF de gramática guaraní
- Implementación de RAG con FAISS
- Configuración de LLMs con OpenRouter
- Evaluación y comparación de modelos

## 1. Instalación de Dependencias

In [ ]:
# Instalación de librerías necesarias
!pip install -q langchain langchain-community
!pip install -q chromadb
!pip install -q sentence-transformers
!pip install -q openai
!pip install -q python-dotenv
!pip install -q transformers torch accelerate
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -q tiktoken
!pip install -q pymupdf  # Para extracción de texto desde PDFs

print("✓ Dependencias instaladas correctamente")

## 2. Configuración e Importaciones

In [ ]:
import os
import json
from typing import List, Dict, Any
from getpass import getpass
import fitz  # PyMuPDF

# LangChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Utilidades
import numpy as np
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

print("✓ Importaciones completadas")

In [ ]:
# Configuración de API Keys (OpenRouter y HuggingFace)
print("Configurando API Keys...")
print("Necesitas:")
print("1. OpenRouter API Key (para GPT-3.5 y Claude)")
print("2. HuggingFace Token (opcional, para modelos privados)")
print()

OPENROUTER_API_KEY = getpass("Ingresa tu API Key de OpenRouter: ")
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

# HuggingFace token (opcional)
HF_TOKEN = getpass("Ingresa tu HuggingFace Token (Enter para omitir): ")
if HF_TOKEN:
    os.environ["HUGGINGFACE_TOKEN"] = HF_TOKEN

# Configuración de dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n✓ Usando dispositivo: {device}")
if device == "cuda":
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memoria disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 3. Carga de Datos - PDF de Gramática Guaraní

En esta sección cargamos el PDF real de gramática guaraní y lo procesamos para crear chunks semánticamente coherentes.

In [ ]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extrae texto de un archivo PDF usando PyMuPDF.
    
    Args:
        pdf_path: Ruta al archivo PDF
        
    Returns:
        Texto extraído del PDF
    """
    try:
        doc = fitz.open(pdf_path)
        text = ""
        
        print(f"Procesando PDF: {pdf_path}")
        print(f"Total de páginas: {len(doc)}")
        
        for page_num, page in enumerate(doc, 1):
            page_text = page.get_text()
            text += f"\n--- Página {page_num} ---\n{page_text}"
            
            if page_num % 10 == 0:
                print(f"  Procesadas {page_num} páginas...")
        
        doc.close()
        print(f"✓ Extracción completada: {len(text)} caracteres")
        return text.strip()
        
    except Exception as e:
        print(f"❌ Error al procesar el PDF: {e}")
        return ""

# Ruta al PDF (ajustar si es necesario)
# En Colab, primero debes subir el archivo o montarlo desde Drive
PDF_PATH = "dataset/GramaticaGuarani.pdf"

# Si estás en Colab, descomenta estas líneas para subir el archivo:
# from google.colab import files
# uploaded = files.upload()
# PDF_PATH = list(uploaded.keys())[0]

# O monta Google Drive:
# from google.colab import drive
# drive.mount('/content/drive')
# PDF_PATH = '/content/drive/MyDrive/path/to/GramaticaGuarani.pdf'

# Extraer texto del PDF
pdf_text = extract_text_from_pdf(PDF_PATH)

# Mostrar muestra del texto extraído
print("\n" + "="*60)
print("MUESTRA DEL TEXTO EXTRAÍDO (primeros 500 caracteres):")
print("="*60)
print(pdf_text[:500])
print("...")

In [ ]:
# Dividir el texto en chunks semánticamente coherentes
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Tamaño de cada chunk en caracteres
    chunk_overlap=200,  # Solapamiento entre chunks para mantener contexto
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # Prioridad de separadores
)

# Crear chunks
text_chunks = text_splitter.split_text(pdf_text)

print(f"✓ Texto dividido en {len(text_chunks)} chunks")
print(f"  Tamaño promedio: {sum(len(chunk) for chunk in text_chunks) / len(text_chunks):.0f} caracteres")
print(f"  Chunk más pequeño: {min(len(chunk) for chunk in text_chunks)} caracteres")
print(f"  Chunk más grande: {max(len(chunk) for chunk in text_chunks)} caracteres")

# Crear documentos de LangChain
documents = []
for i, chunk in enumerate(text_chunks):
    documents.append(
        Document(
            page_content=chunk,
            metadata={
                "source": "GramaticaGuarani.pdf",
                "chunk_id": i
            }
        )
    )

print(f"\n✓ Creados {len(documents)} documentos de LangChain")

# Mostrar ejemplo de un chunk
print("\n" + "="*60)
print("EJEMPLO DE CHUNK #5:")
print("="*60)
print(documents[5].page_content[:300] + "...")

## 4. Implementación del Sistema RAG

In [ ]:
# Crear embeddings multilingües
print("Cargando modelo de embeddings multilingüe...")
print("Modelo: sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
print("Este modelo soporta 50+ idiomas, incluyendo español y guaraní.\n")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    model_kwargs={'device': device}
)

print("✓ Modelo de embeddings cargado")

In [ ]:
# Crear vector store con FAISS
print("Creando vector store con FAISS...")
print(f"Procesando {len(documents)} documentos...\n")

vectorstore = FAISS.from_documents(documents, embeddings)

# Guardar el vector store para uso posterior
vectorstore.save_local("vectorstore_guarani")

print("✓ Vector store creado y guardado en 'vectorstore_guarani/'")
print("  Este vector store se usará en la aplicación Chainlit")

In [ ]:
# Probar el retriever con una consulta de ejemplo
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Recuperar los 3 documentos más relevantes
)

print("✓ Retriever configurado\n")

# Prueba de recuperación
test_query = "¿Cuáles son los pronombres personales en guaraní?"
print(f"Consulta de prueba: {test_query}\n")

relevant_docs = retriever.get_relevant_documents(test_query)

print(f"Documentos recuperados: {len(relevant_docs)}\n")
for i, doc in enumerate(relevant_docs, 1):
    print(f"--- Documento {i} ---")
    print(f"Fuente: {doc.metadata['source']}")
    print(f"Chunk ID: {doc.metadata['chunk_id']}")
    print(f"Contenido (primeros 200 caracteres):\n{doc.page_content[:200]}...\n")

## 5. Configuración de LLMs con OpenRouter

In [ ]:
import requests

class OpenRouterLLM:
    """Wrapper para usar OpenRouter con diferentes modelos"""
    
    def __init__(self, model_name: str, api_key: str):
        self.model_name = model_name
        self.api_key = api_key
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"
    
    def __call__(self, prompt: str, **kwargs) -> str:
        """Genera respuesta del modelo"""
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        data = {
            "model": self.model_name,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "max_tokens": kwargs.get("max_tokens", 500),
            "temperature": kwargs.get("temperature", 0.7)
        }
        
        try:
            response = requests.post(self.api_url, headers=headers, json=data, timeout=30)
            
            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]
            else:
                error_msg = f"Error {response.status_code}: {response.text}"
                print(f"❌ {error_msg}")
                return error_msg
        except Exception as e:
            error_msg = f"Error de conexión: {str(e)}"
            print(f"❌ {error_msg}")
            return error_msg
    
    def generate(self, prompt: str, **kwargs) -> str:
        """Alias para compatibilidad"""
        return self.__call__(prompt, **kwargs)

# Configurar dos modelos diferentes para comparación
print("Configurando modelos LLM...\n")

# Modelo 1: GPT-3.5 Turbo (más rápido y económico)
model_1 = OpenRouterLLM(
    model_name="openai/gpt-3.5-turbo",
    api_key=OPENROUTER_API_KEY
)

# Modelo 2: Claude Sonnet (más potente)
model_2 = OpenRouterLLM(
    model_name="anthropic/claude-3.5-sonnet",
    api_key=OPENROUTER_API_KEY
)

print("✓ Modelos configurados:")
print("  - Modelo 1: GPT-3.5 Turbo (OpenAI)")
print("  - Modelo 2: Claude 3.5 Sonnet (Anthropic)")

# Probar conexión con ambos modelos
print("\nProbando conexión con los modelos...\n")

test_prompt = "Di 'hola' en una palabra."
print(f"Prompt de prueba: {test_prompt}\n")

print("Modelo 1 (GPT-3.5):")
response_1 = model_1.generate(test_prompt, max_tokens=50)
print(f"  Respuesta: {response_1}\n")

print("Modelo 2 (Claude 3.5):")
response_2 = model_2.generate(test_prompt, max_tokens=50)
print(f"  Respuesta: {response_2}\n")

if "Error" not in response_1 and "Error" not in response_2:
    print("✓ Ambos modelos funcionan correctamente")
else:
    print("⚠️ Verifica tus API keys y conexión a internet")

## 6. Implementación de Diferentes Estrategias

In [ ]:
class GuaraniChatbot:
    """Sistema de chatbot para guaraní con diferentes estrategias"""
    
    def __init__(self, llm, retriever=None):
        self.llm = llm
        self.retriever = retriever
    
    def zero_shot(self, question: str) -> str:
        """Estrategia Zero-Shot: sin ejemplos ni contexto adicional"""
        prompt = f"""Eres un asistente experto en el idioma guaraní.
        
Pregunta: {question}

Responde de manera clara y concisa."""
        
        return self.llm.generate(prompt)
    
    def few_shot(self, question: str) -> str:
        """Estrategia Few-Shot: con ejemplos de referencia"""
        prompt = f"""Eres un asistente experto en el idioma guaraní. Aquí hay algunos ejemplos:

Ejemplo 1:
Pregunta: ¿Cómo se dice "yo" en guaraní?
Respuesta: En guaraní, "yo" se dice "Che".

Ejemplo 2:
Pregunta: ¿Cómo se dice "hola" en guaraní?
Respuesta: En guaraní, una forma común de saludar es "Mba'éichapa", que significa "¿cómo estás?".

Ejemplo 3:
Pregunta: ¿Cuál es la estructura básica de las oraciones en guaraní?
Respuesta: El guaraní usa una estructura SOV (Sujeto-Objeto-Verbo). Por ejemplo: "Che tembi'u ajuka" (Yo comida como).

Ahora responde esta pregunta:
Pregunta: {question}
Respuesta:"""
        
        return self.llm.generate(prompt)
    
    def rag(self, question: str) -> str:
        """Estrategia RAG: con recuperación de documentos relevantes"""
        if self.retriever is None:
            return "Error: Retriever no configurado"
        
        # Recuperar documentos relevantes
        relevant_docs = self.retriever.get_relevant_documents(question)
        
        # Construir contexto
        context = "\n\n".join([doc.page_content for doc in relevant_docs])
        
        prompt = f"""Eres un asistente experto en el idioma guaraní. Usa la siguiente información de referencia para responder la pregunta.

CONTEXTO:
{context}

PREGUNTA: {question}

INSTRUCCIONES:
- Basa tu respuesta en la información del contexto proporcionado
- Si la información no está en el contexto, indícalo claramente
- Responde de manera clara y educativa
- Si es apropiado, incluye ejemplos en guaraní

RESPUESTA:"""
        
        return self.llm.generate(prompt)

print("✓ Clase GuaraniChatbot implementada")

## 7. Evaluación y Comparación

In [ ]:
# Preguntas de prueba más específicas basadas en gramática guaraní
test_questions = [
    "¿Cuáles son los pronombres personales en guaraní?",
    "¿Cómo se dice 'nosotros' en guaraní y cuál es la diferencia entre las formas inclusiva y exclusiva?",
    "¿Cuál es el orden típico de las palabras en una oración guaraní?",
    "¿Cómo se conjuga el verbo 'ir' en primera, segunda y tercera persona?",
    "¿Cuáles son los saludos más comunes en guaraní?",
    "¿Qué son los prefijos personales en guaraní y cómo se usan?",
    "¿Cómo se forma el plural en guaraní?",
    "¿Cuáles son las características principales de la fonología del guaraní?"
]

print("✓ Preguntas de evaluación preparadas:")
print(f"  Total: {len(test_questions)} preguntas\n")
for i, q in enumerate(test_questions, 1):
    print(f"  {i}. {q}")

In [ ]:
import time

def evaluate_model(model, model_name: str, retriever=None):
    """Evalúa un modelo con las tres estrategias"""
    
    chatbot = GuaraniChatbot(model, retriever)
    results = {
        "model": model_name,
        "strategies": {}
    }
    
    print(f"\n{'='*60}")
    print(f"EVALUANDO: {model_name}")
    print(f"{'='*60}\n")
    
    for strategy_name in ["zero_shot", "few_shot", "rag"]:
        print(f"\n--- Estrategia: {strategy_name.upper()} ---\n")
        strategy_results = []
        
        for i, question in enumerate(test_questions, 1):
            print(f"Pregunta {i}/{len(test_questions)}: {question}")
            
            start_time = time.time()
            
            if strategy_name == "zero_shot":
                answer = chatbot.zero_shot(question)
            elif strategy_name == "few_shot":
                answer = chatbot.few_shot(question)
            else:  # rag
                answer = chatbot.rag(question)
            
            elapsed_time = time.time() - start_time
            
            print(f"Respuesta ({elapsed_time:.2f}s): {answer[:150]}...\n")
            
            strategy_results.append({
                "question": question,
                "answer": answer,
                "response_time": elapsed_time
            })
            
            # Pequeña pausa para no saturar la API
            time.sleep(1)
        
        results["strategies"][strategy_name] = strategy_results
        
        avg_time = sum(r["response_time"] for r in strategy_results) / len(strategy_results)
        print(f"\n✓ Estrategia {strategy_name.upper()} completada")
        print(f"  Tiempo promedio de respuesta: {avg_time:.2f}s")
    
    return results

print("✓ Función de evaluación lista")

In [ ]:
# Evaluar Modelo 1 (GPT-3.5 Turbo)
print("\n" + "#"*60)
print("# INICIANDO EVALUACIÓN DEL MODELO 1")
print("#"*60)

results_model_1 = evaluate_model(model_1, "GPT-3.5 Turbo", retriever)

print("\n" + "="*60)
print("✓ EVALUACIÓN DEL MODELO 1 COMPLETADA")
print("="*60)

In [ ]:
# Evaluar Modelo 2 (Claude 3.5 Sonnet)
print("\n" + "#"*60)
print("# INICIANDO EVALUACIÓN DEL MODELO 2")
print("#"*60)

results_model_2 = evaluate_model(model_2, "Claude 3.5 Sonnet", retriever)

print("\n" + "="*60)
print("✓ EVALUACIÓN DEL MODELO 2 COMPLETADA")
print("="*60)

In [ ]:
# Guardar resultados
import json

all_results = {
    "model_1": results_model_1,
    "model_2": results_model_2,
    "metadata": {
        "pdf_source": "GramaticaGuarani.pdf",
        "total_questions": len(test_questions),
        "embedding_model": "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
        "vector_store": "FAISS",
        "chunk_size": 1000,
        "chunk_overlap": 200
    }
}

with open("evaluation_results.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

print("✓ Resultados guardados en 'evaluation_results.json'")
print(f"  Tamaño del archivo: {os.path.getsize('evaluation_results.json') / 1024:.2f} KB")

## 8. Análisis de Resultados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def analyze_results(all_results):
    """Analiza y visualiza los resultados de la evaluación"""
    
    print("\n" + "="*60)
    print("ANÁLISIS DE RESULTADOS")
    print("="*60 + "\n")
    
    # Análisis cuantitativo
    comparison_data = []
    
    for model_key in ["model_1", "model_2"]:
        model_data = all_results[model_key]
        model_name = model_data["model"]
        
        print(f"\n{model_name}:")
        print("-" * 40)
        
        for strategy_name, strategy_results in model_data["strategies"].items():
            # Calcular métricas
            avg_length = sum(len(r["answer"]) for r in strategy_results) / len(strategy_results)
            avg_time = sum(r["response_time"] for r in strategy_results) / len(strategy_results)
            
            print(f"\n  Estrategia: {strategy_name.upper()}")
            print(f"    Total de respuestas: {len(strategy_results)}")
            print(f"    Longitud promedio: {avg_length:.0f} caracteres")
            print(f"    Tiempo promedio: {avg_time:.2f} segundos")
            
            comparison_data.append({
                "Modelo": model_name,
                "Estrategia": strategy_name,
                "Longitud Promedio": avg_length,
                "Tiempo Promedio": avg_time
            })
    
    # Crear DataFrame para visualización
    df = pd.DataFrame(comparison_data)
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Gráfico 1: Longitud de respuestas
    df_pivot = df.pivot(index="Estrategia", columns="Modelo", values="Longitud Promedio")
    df_pivot.plot(kind="bar", ax=axes[0], rot=0)
    axes[0].set_title("Longitud Promedio de Respuestas", fontsize=14, fontweight="bold")
    axes[0].set_ylabel("Caracteres")
    axes[0].set_xlabel("Estrategia")
    axes[0].legend(title="Modelo")
    axes[0].grid(axis="y", alpha=0.3)
    
    # Gráfico 2: Tiempo de respuesta
    df_pivot_time = df.pivot(index="Estrategia", columns="Modelo", values="Tiempo Promedio")
    df_pivot_time.plot(kind="bar", ax=axes[1], rot=0, color=["#FF6B6B", "#4ECDC4"])
    axes[1].set_title("Tiempo Promedio de Respuesta", fontsize=14, fontweight="bold")
    axes[1].set_ylabel("Segundos")
    axes[1].set_xlabel("Estrategia")
    axes[1].legend(title="Modelo")
    axes[1].grid(axis="y", alpha=0.3)
    
    plt.tight_layout()
    plt.savefig("evaluation_comparison.png", dpi=300, bbox_inches="tight")
    plt.show()
    
    print("\n" + "="*60)
    print("CONCLUSIONES Y PRÓXIMOS PASOS")
    print("="*60)
    print("""
Para un análisis completo, considera evaluar:

1. PRECISIÓN: ¿Las respuestas son correctas según la gramática guaraní?
   → Revisar manualmente las respuestas comparándolas con el PDF

2. RELEVANCIA: ¿Las respuestas abordan directamente la pregunta?
   → Verificar que las respuestas respondan lo que se pregunta

3. COMPLETITUD: ¿Las respuestas proporcionan información suficiente?
   → Evaluar si incluyen ejemplos y explicaciones adecuadas

4. CONSISTENCIA: ¿El modelo es consistente en sus respuestas?
   → Comparar respuestas similares entre estrategias

Compara especialmente:
- Zero-shot vs Few-shot: ¿Los ejemplos mejoran el rendimiento?
- Few-shot vs RAG: ¿El contexto recuperado es más útil que los ejemplos fijos?
- Modelo 1 vs Modelo 2: ¿Qué modelo maneja mejor el idioma de bajo recursos?
- RAG con PDF vs sin PDF: ¿El RAG realmente beneficia al guaraní?

Gráfico guardado en: evaluation_comparison.png
    """)
    
    return df

In [ ]:
# Ejecutar análisis
df_results = analyze_results(all_results)

# Mostrar tabla de comparación
print("\n" + "="*60)
print("TABLA DE COMPARACIÓN")
print("="*60)
print(df_results.to_string(index=False))

## 9. Exportar Modelos y Configuración

In [ ]:
# Descargar archivos necesarios para la aplicación Chainlit
print("Preparando archivos para descarga...\n")

# Comprimir el vector store
!zip -r vectorstore_guarani.zip vectorstore_guarani/

print("\n" + "="*60)
print("ARCHIVOS GENERADOS")
print("="*60)
print("\n1. vectorstore_guarani.zip")
print("   - Contiene el vector store FAISS con embeddings del PDF")
print("   - Descomprimir en el directorio del proyecto local")
print("\n2. evaluation_results.json")
print("   - Resultados completos de la evaluación")
print("   - Incluye respuestas de ambos modelos con las 3 estrategias")
print("\n3. evaluation_comparison.png")
print("   - Gráficos comparativos de rendimiento")
print("="*60)

# Si estás en Colab, descarga los archivos
try:
    from google.colab import files
    
    print("\nDescargando archivos...")
    print("1. Vector store (vectorstore_guarani.zip)")
    files.download('vectorstore_guarani.zip')
    
    print("2. Resultados de evaluación (evaluation_results.json)")
    files.download('evaluation_results.json')
    
    print("3. Gráficos de comparación (evaluation_comparison.png)")
    files.download('evaluation_comparison.png')
    
    print("\n✓ Archivos descargados correctamente")
except ImportError:
    print("\n⚠️ No estás en Google Colab")
    print("Los archivos están disponibles en el directorio actual")

print("\n" + "="*60)
print("PRÓXIMOS PASOS")
print("="*60)
print("""
1. Descargar los archivos generados
2. Descomprimir vectorstore_guarani.zip en tu proyecto local
3. Configurar el archivo .env con tu OPENROUTER_API_KEY
4. Ejecutar la aplicación Chainlit:
   
   cd /ruta/a/tu/proyecto
   chainlit run app.py -w
   
5. Probar el chatbot con preguntas en guaraní
6. Usar /rag on y /rag off para comparar modos

¡El sistema RAG para guaraní está listo! 🇵🇾
""")